# Right Left up down spatial 15

In [2]:
import os
from pathlib import Path
import pickle

import numpy as np
# import remotezip as rz
# import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

from frame_generator import FrameGenerator
from model import create_model, visualise_model

In [3]:
DATA_PATH = "data_15_right_left_up_down_1200"
NOTEBOOK_NAME = "2_2d_plus_1_rlud_sp15_1200_n_step_2"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

# PARAMS
# number of frames taken from each video
n_frames = 36
# number of frames skipped from each video
frame_step = 2
batch_size = 8
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


In [4]:
from validate_files import get_video_lengths

lengths_array = get_video_lengths(f"./{DATA_PATH}/test/")
np.append(lengths_array, get_video_lengths(f"./{DATA_PATH}/train/"))
np.append(lengths_array, get_video_lengths(f"./{DATA_PATH}/validation/"))

# Calculate average
average = np.mean(lengths_array)

# Calculate maximum
maximum = np.max(lengths_array)

# Calculate minimum
minimum = np.min(lengths_array)

print("Average:", average)
print("Maximum:", maximum)
print("Minimum:", minimum)

Average: 46.21875
Maximum: 71
Minimum: 30


## Preprocess video data

Load something something data tf.data.Dataset

In [5]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['val'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        training=False
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['test'],
        n_frames=n_frames,
        index_df=index_df,
        frame_step=frame_step,
        training=False
    ),
    output_signature = output_signature
)

print(type(test_ds))

# Batch the data
test_ds = test_ds.batch(batch_size)

print(type(test_ds))


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


## Model Creation

In [6]:
model = create_model(n_frames=n_frames, height=HEIGHT, width=WIDTH)

## Validate Model and Data

In [7]:
# testing tensor is setup correct
iter(train_ds)

## Build Model

In [8]:
frames, label = next(iter(train_ds))

In [9]:
model.build(frames)

## Visualise Model

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 36, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 conv2_plus1d (Conv2Plus1D)     (None, 36, 224, 224  3152        ['input_1[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 36, 224, 224  64         ['conv2_plus1d[0][0]']           
 alization)                     , 16)                                                         

In [11]:
visualise_model(
    model=model,
    file_path=DATA_PATH + "/model_summary.dot"
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


## Load the Model

Using BinaryCrossentropy as it is more effective for binary data

from_logits is false because final layer includes a sigmoid activation,

In [12]:
previous_runs = 2

In [13]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                    # tf.keras.metrics.Precision(),
                    # tf.keras.metrics.Recall()
                    # tf.keras.metrics.Precision(class_id=0, name='precision_neg'),
                    # tf.keras.metrics.Precision(class_id=1, name='precision_pos'),
                    # tf.keras.metrics.Recall(class_id=0, name='recall_neg'),
                    # tf.keras.metrics.Recall(class_id=1, name='recall_pos')
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,  # Save only the best model based on a monitored metric (e.g., val_loss), will only replace saved value if it is better
    monitor='val_loss',
    mode='min',  # 'min' for loss, 'max' for accuracy
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [14]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

loading model from weights: data_15_right_left_up_down_1200/2_2d_plus_1_rlud_sp15_1200_n_step_2/model-runs-2-cp-11-0.35.ckpt


## Train the Model

In [ ]:
previously_run_epochs = 0 + 13 + 12
history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/25
480/480 [==============================] - 1608s 3s/step - loss: 0.4101 - accuracy: 0.8505 - val_loss: 0.4608 - val_accuracy: 0.8208
Epoch 2/25
480/480 [==============================] - 1645s 3s/step - loss: 0.3981 - accuracy: 0.8521 - val_loss: 0.3574 - val_accuracy: 0.8687
Epoch 3/25
480/480 [==============================] - 1612s 3s/step - loss: 0.3899 - accuracy: 0.8612 - val_loss: 0.3479 - val_accuracy: 0.8792
Epoch 4/25
480/480 [==============================] - 1644s 3s/step - loss: 0.3623 - accuracy: 0.8703 - val_loss: 0.3408 - val_accuracy: 0.8750
Epoch 5/25
480/480 [==============================] - 1618s 3s/step - loss: 0.3463 - accuracy: 0.8797 - val_loss: 0.3402 - val_accuracy: 0.8813
Epoch 6/25
480/480 [==============================] - 1648s 3s/step - loss: 0.3334 - accuracy: 0.8781 - val_loss: 0.3468 - val_accuracy: 0.8771
Epoch 7/25
480/480 [==============================] - 1626s 3s/step - loss: 0.3163 - accuracy: 0.8891 - val_loss: 0.3368 - val_accuracy:

## Analyse results

In [18]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

for batch in test_ds:
    x, y = batch
    true_labels.extend(y.numpy())
    preds = model.predict(x)
    preds = softmax(preds, axis=1)
    preds = np.argmax(preds, axis=1)
    predictions.extend(preds)

true_labels = np.array(true_labels)
predictions = np.array(predictions)

2023-12-24 13:53:44.384908: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.
2023-12-24 13:53:45.818856: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


1/1 [==============================] - 7s 7s/step


2023-12-24 13:53:53.122601: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.
2023-12-24 13:53:53.258134: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 173408256 exceeds 10% of free system memory.


1/1 [==============================] - 1s 893ms/step


In [19]:
fg = FrameGenerator(
    subset_paths['test'],
    n_frames=n_frames,
    index_df=index_df,
    frame_step=frame_step,
    training=False
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

true_labels = np.array(true_labels)
predictions = np.array(predictions)

accuracy = accuracy_score(true_labels, predictions)

precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

Overall Accuracy: 0.9166666666666666
Class Pushing something from left to right: Precision: 0.9661016949152542, Recall: 0.95, F1 Score: 0.957983193277311
Class Pushing something from right to left: Precision: 0.9734513274336283, Recall: 0.9166666666666666, F1 Score: 0.944206008583691
Class Moving something up: Precision: 0.8296296296296296, Recall: 0.9333333333333333, F1 Score: 0.8784313725490196
Class Moving something down: Precision: 0.9122807017543859, Recall: 0.8666666666666667, F1 Score: 0.8888888888888888
